### Bathe-Ansatz at Finite temperature
-- Evaluated at half-filling and $S_z = 0$.
      (Referece: https://journals.jps.jp/doi/10.1143/JPSJ.59.1357)

#### NOTE that U = U/4 here (we first keep the scaling)

The grand potential per site is evaluated by

\begin{equation}
\Omega = e_0 - \mu - \int_{-\pi}^{\pi} \rho_0(k)G[\kappa(k)]dk
 - \int_{-\infty}^{\infty} \sigma_0(\Lambda)[\varepsilon_1^{-}(\Lambda) - T\ln b_1^{-}]d\Lambda
\end{equation}
where $e_0$, $\rho_0(k)$, $\sigma_0(\Lambda)$ are ground state quantities that can be evaluated from ground state BA.

The other quantities in the integrations are defined as following

\begin{equation}
G(y) = T\ln [1+\exp(y/T)], \hspace{0.5cm} b_n^{\pm} = \frac{\sinh^2 x^{\pm}}{\sinh^2 (n+1)x^{\pm}}  \hspace{0.3cm} \left(=\frac{1}{(n+1)^2} \text{ if } x^{\pm} = 0\right)
\end{equation}

\begin{equation}
\kappa(k) = \kappa_0(k) + T\ln\left(\frac{\cosh x^+}{\cosh x^-}\right) + \int_{-\infty}^{\infty}s(\Lambda-\sin k)
[\varepsilon^+_1(\Lambda)-\varepsilon^-_1(\Lambda)] d\Lambda
\end{equation}

where
\begin{equation}
\kappa_0(k) = -2\cos k - 4\int_{-\infty}^{\infty} s(\Lambda -\sin k)\Re [1 - (\Lambda - iU)^2]^{1/2} d\Lambda
\end{equation}

\begin{equation}
\varepsilon_n^{\pm}(\Lambda) = T\ln \{b_n^{\pm}+(1-b_n^{\pm})\exp[E_n^{\pm}(\Lambda)/T]\} \hspace{0.3cm} \text{for} \hspace{0.3cm} n\geq 1
\end{equation}

\begin{equation}
E_1^{\pm}(\Lambda) = s*\varepsilon_2^{\pm}(\Lambda) - \int_{-\pi}^{\pi} \cos(k)  s(\Lambda - \sin k)G[\pm \kappa(k)]d k
\end{equation}

and 
\begin{equation}
E_n^{\pm}(\Lambda) = s*(\varepsilon_{n-1}^{\pm} + \varepsilon_{n+1}^{\pm}) \hspace{0.5cm} \text{for} \hspace{0.5cm} n\geq 2
\end{equation}

\begin{equation}
x^+ = (2U-\mu)/T \hspace{0.3cm} (=0 \text{ if half-filling} ), \hspace{0.5cm} x^- = 0
\end{equation}
and
\begin{equation}
s*f = \int_{-\infty}^{\infty} s(\Lambda - \Lambda')f(\Lambda')d\Lambda'
\end{equation}

#### Asymptotic conditions
\begin{equation}
\lim_{\Lambda \rightarrow \infty}\varepsilon_n^{\pm}(\Lambda) = 0
\end{equation}

\begin{equation}
\lim_{n\rightarrow \infty} \left[\varepsilon^{\pm}_{n+1}(\Lambda) - \int_{\infty}^{\infty} \varepsilon_n^{\pm}(\Lambda') \frac{U/\pi}{(\Lambda' - \Lambda)^2 + U^2}d\Lambda'\right] = 0
\end{equation}
which gives $\epsilon_n^{\pm}\rightarrow 0$ as $n\rightarrow \infty$. $n=12$ is quiet enough.

In [91]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

### Implementation
#### 1. Solve ground state
half-filling is much easier with the Bessel functions

#### 2. Solve for $\varepsilon_n^{\pm}$, $E_n^{\pm}$ and $\kappa(k)$.

First try: 
1. Initial guess for $E_1^{\pm}$.
2. Solve for $\varepsilon_n^{\pm}$, $E_n^{\pm}$ with $n = 2, ..., N_m$ assuming $\varepsilon_{n+1}^{\pm}=0$.
3. Solve back with the knowledge of new $\varepsilon_n^{\pm}$, $E_n^{\pm}$.
4. Compare $\varepsilon_1^{\pm}$ to see if convergence reached.


In [192]:
class BetheAnsatzFT(object):
    # half-filling, Sz = 0
    def __init__(self, U, T, ngrid):
        self.U = U
        self.T = T
        self.ngrid = ngrid
        self.Q = np.pi
        self.B = 10
        self.kgrid = np.zeros(ngrid)
        self.lgrid = np.zeros(ngrid)
        self.dk = 2.*self.Q / ngrid
        self.dl = 2.*self.B / ngrid
        #some functions used in integration functions
        self.kappa0 = np.zeros(ngrid)
        self.kappa = np.zeros(ngrid)
    
    def generate_grids(self):
        # symmetrized partition
        self.kgrid = np.linspace(-self.Q, self.Q, self.ngrid, endpoint=False)
        self.lgrid = np.linspace(-self.B, self.B, self.ngrid, endpoint=False)
        self.kgrid += self.dk/2.
        self.lgrid += self.dl/2.
    def sfunc(self, l, k):
        self.s = 1./(4.*self.U*np.cosh(np.pi*(l-k)/(2*self.U)))
        return self.s
    def kappa0func(self):
        kappa0 = np.zeros(self.ngrid)
        i = 0
        for k in self.kgrid:
            kappa0[i] = -2.*np.cos(k) - 4.*self.dl*np.dot(self.sfunc(self.lgrid,np.sin(k)),np.sqrt(1.-(self.lgrid-1.j*self.U)**2).real)
            i+=1
        self.kappa0 = kappa0
        return kappa0
    def Gfunc(self, y):
        return self.T * np.log(1.+np.exp(y/self.T))
    def sconvfunc(self, fl):
        convf = np.zeros(self.ngrid)
        i = 0
        for l in self.lgrid:
            convf[i] = self.dl*np.dot(self.sfunc(l, self.lgrid), fl)
            i += 1
        return convf
    # functions to be used in self-consistency
    def kappafunc(self, eps1p, eps1m):
        kappa = np.zeros(self.ngrid)
        self.kappa0 = self.kappa0func()
        i = 0
        for k in self.kgrid:
            kappa[i] = self.dl * np.dot(self.sfunc(self.lgrid, np.sin(k)), eps1p-eps1m)
            i += 1
        self.kappa = kappa + self.kappa0
        return self.kappa
    
    def epsnfunc(self, n, En):
        bn = 1./((n+1.)**2)
        return self.T * np.log(bn + (1-bn)*np.exp(En/self.T))
    def E1func(self, eps2, p=1.):
        # p : parity
        E1 = np.zeros(self.ngrid)
        i = 0
        for l in self.lgrid:
            E1[i] = -self.dk * np.dot(np.cos(self.kgrid)*self.sfunc(l,np.sin(self.kgrid)), self.Gfunc(p*self.kappa))
            i += 1

        E1 += self.sconvfunc(eps2)
        return E1
    def Enfunc(self, epsm, epsp):
        return self.sconvfunc(epsm + epsp)
   
    def solve_epsilon_scf(self, nstep=20, tol = 1e-5):
        #E1 = np.random.randn(self.ngrid)*0.5
        nmax  = 12
        ngrid = self.ngrid
        Ep   = np.zeros((nmax+1, ngrid))
        Em   = np.zeros((nmax+1, ngrid))
        epsp = np.zeros((nmax+1, ngrid))
        epsm = np.zeros((nmax+1, ngrid))
        # initial guess: eps1p = eps1m, high orders are zero
        kappa = self.kappa.copy()
        for itr in range(nstep):
            self.kappa = self.kappafunc(epsp[0], epsm[0])
            Ep[0] = self.E1func(epsp[1], 1.)
            Em[0] = self.E1func(epsm[1], -1.)
            epsp[0] = self.epsnfunc(1, Ep[0])
            epsm[0] = self.epsnfunc(1, Em[0])
            for i in range(1, nmax):
                Ep[i] = self.sconvfunc(epsp[i-1]+epsp[i+1])
                Em[i] = self.sconvfunc(epsm[i-1]+epsm[i+1])
                epsp[i] = self.epsnfunc(i+1, Ep[i])
                epsm[i] = self.epsnfunc(i+1, Em[i])
            diff = np.linalg.norm(self.kappa - kappa)
            if diff < tol:
                print "The convergence is achieved after %d loops!"%(itr+1), "Final diff:", diff
                break
            else:
                kappa = self.kappa.copy()
        if(itr == nstep):
            print "The convergence is NOT acheived after %d loops"%nstep
        print epsm

In [193]:
baobj = BetheAnsatzFT(4,1,10)
baobj.generate_grids()
baobj.solve_epsilon_scf()

The convergence is achieved after 16 loops! Final diff: 8.3396817384e-06
[[ -2.93711343e-02  -5.61551144e-02  -1.05333245e-01  -1.90308215e-01
   -2.86596802e-01  -2.86596802e-01  -1.90308215e-01  -1.05333245e-01
   -5.61551144e-02  -2.93711343e-02]
 [ -2.96558820e-02  -4.83832396e-02  -7.08796985e-02  -9.37563765e-02
   -1.09229753e-01  -1.09229753e-01  -9.37563765e-02  -7.08796985e-02
   -4.83832396e-02  -2.96558820e-02]
 [ -2.03332016e-02  -3.08315221e-02  -4.09851765e-02  -4.93001529e-02
   -5.40788138e-02  -5.40788138e-02  -4.93001529e-02  -4.09851765e-02
   -3.08315221e-02  -2.03332016e-02]
 [ -1.24651037e-02  -1.83766537e-02  -2.35794562e-02  -2.74656916e-02
   -2.95570107e-02  -2.95570107e-02  -2.74656916e-02  -2.35794562e-02
   -1.83766537e-02  -1.24651037e-02]
 [ -7.40791583e-03  -1.08094948e-02  -1.36944031e-02  -1.57714743e-02
   -1.68600574e-02  -1.68600574e-02  -1.57714743e-02  -1.36944031e-02
   -1.08094948e-02  -7.40791583e-03]
 [ -4.37609428e-03  -6.36198941e-03  -8.02

In [81]:
1./(13**2)

0.005917159763313609

In [ ]:
exp()

In [129]:
np.arange(3)*(np.arange(3)+1)

array([0, 2, 6])